<video width="1000" controls>
  <source src="src/nn_lm_prob_idea.mp4" type="video/mp4">
</video>

In [ ]:
# !pip install transformers accelerate datasets

# Base PRE-TRAINED model

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

# Modelli a confronto
BASE_MODEL = "google/gemma-3-1b-pt"
INSTRUCT_MODEL = "google/gemma-3-1b-it"

# Carichiamo tokenizer condiviso
tokenizer = AutoTokenizer.from_pretrained(INSTRUCT_MODEL)

# Carichiamo i due modelli
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

def generate(model, prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

# Prompt di test (tipici instruction-following)
prompts = [
    "Spiega cos'è il machine learning a un bambino di 10 anni.",
    "Fornisci tre vantaggi delle auto elettriche in formato elenco.",
    "Scrivi un breve riassunto del romanzo 'Il piccolo principe'."
]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [2]:
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16).to(device)

for p in prompts:
    print("=" * 80)
    print(f"[PROMPT]\n{p}\n")
    print("[RISPOSTA MODELLO BASE]:\n", generate(base_model, p))

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
2025-10-27 16:08:21.612271: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[PROMPT]
Spiega cos'è il machine learning a un bambino di 10 anni.

[RISPOSTA MODELLO BASE]:
 <bos>Spiega cos'è il machine learning a un bambino di 10 anni.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
[PROMPT]
Fornisci tre vantaggi delle auto elettriche in formato elenco.

[RISPOSTA MODELLO BASE]:
 <bos>Fornisci tre vantaggi delle auto elettriche in formato elenco.<pad><pad><pad><pad><pad><pad><pad><p

# INSTRUCTION-TUNED model

In [3]:
inst_model = AutoModelForCausalLM.from_pretrained(INSTRUCT_MODEL, torch_dtype=torch.float16).to(device)

for p in prompts:
    print("=" * 80)
    print(f"[PROMPT]\n{p}\n")
    print("\n[RISPOSTA MODELLO ISTRUZIONATA]:\n", generate(inst_model, p))

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

[PROMPT]
Spiega cos'è il machine learning a un bambino di 10 anni.


[RISPOSTA MODELLO ISTRUZIONATA]:
 <bos>Spiega cos'è il machine learning a un bambino di 10 anni.

Immagina di avere un cane che impara a sedersi. All'inizio, il cane non sa cosa fare. Ma tu gli dici "seduto" e gli dai un premio quando si siede. Dopo un po', il cane impara a sedersi quasi sempre!

Il machine learning è un po' come questo. Invece di insegnare a un cane a sedersi, insegniamo a un computer a fare cose.

*   **Dati:**  Come quando insegni al cane, usiamo dei "dati". Questi sono esempi di cosa vogliamo che il computer impari. Ad esempio,
[PROMPT]
Fornisci tre vantaggi delle auto elettriche in formato elenco.


[RISPOSTA MODELLO ISTRUZIONATA]:
 <bos>Fornisci tre vantaggi delle auto elettriche in formato elenco.

1.  **Emissioni ridotte:** Le auto elettriche non producono emissioni di scarico, contribuendo a migliorare la qualità dell'aria e a ridurre l'impatto ambientale.
2.  **Costi di esercizio inferiori:*

In [5]:
# Selezione dispositivi
dtype_base = torch.float32

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# Evita loop su <pad>
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=dtype_base).to(device)

gen_cfg_sample = GenerationConfig(
    max_new_tokens=300,
    do_sample=True,
    temperature=0.9,
    top_p=0.92,
    top_k=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

prompts_good = [
    "Il machine learning è una branca dell'intelligenza artificiale che",
]

def generate(model, prompt, device, gen_cfg):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(**inputs, generation_config=gen_cfg)
    return tokenizer.decode(out[0], skip_special_tokens=False)

for p in prompts_good:
    print("="*80)
    print("[PROMPT]\n", p, "\n")
    print("[BASE CONTINUATION]\n", generate(base, p, device, gen_cfg_sample))


[PROMPT]
 Il machine learning è una branca dell'intelligenza artificiale che 

[BASE CONTINUATION]
 <bos>Il machine learning è una branca dell'intelligenza artificiale che ha lo scopo di sviluppare un'apprendimento automatico attraverso l'analisi del dato.

Perché i dati siano accurati, è necessario un buon modello predittivo che non deve essere "visto" ma "sentito" dall'algoritmo. Questo significa che un modello predittivo basato sulla teoria della probabilità deve avere la capacità di valutare le probabilità e i conflitti delle diverse interpretazioni di dati.

Uno degli aspetti chiave per sviluppare un buon modello predittivo è l'accuratezza. Un buon modello predittivo dovrebbe avere un'alta precisione, ma anche una bassa specificità. A seconda del problema, ad esempio, un buon modello predittivo deve essere in grado di identificare diversi elementi, ma non tutti.

Quando si costruisce un modello predittivo, è necessario che il modello predittivo abbia una certa accuratezza. Se il m